# Optimizing Album Sales for Chinook Music Store

In this project, I will be using SQL to organize the data from the Chinook database. The Chinook database is a sample database available for SQL that is commonly used for running demos. The Chinook database is supposed to imitate sales data from on online store that sells music around the world. The structure of the Chinook database can be seen on the diagram below.
<img src="Chinook.png">

Before analyzing the dataset, I will load SQL and connect to the Chinook database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Next, I will list all of the tables in the dataset to make sure they match those indicated in the diagram above.

In [2]:
%%sql
SELECT
    name,
    type
  FROM sqlite_master
 WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


These table names match those shown in the Chinook diagram, so I can begin analyzing the data.

## 1. Most Popular Genres

In this section, I will find the number of tracks sold in the Chinook store in each genre to help the owner of the Chinook store determine which types of music he or she should focus on selling.

In [3]:
%%sql
WITH -- Selecting invoice information from tracks sold in the USA
    usa_tracks AS
    (
     SELECT
        il.*
       FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     WHERE c.country = "USA"
    )

SELECT
    g.name genre,
    COUNT(ut.invoice_line_id) tracks_sold,
    -- Calculates the percent of total tracks sold are within each given genre
    ROUND(CAST(COUNT(ut.invoice_line_id) AS FLOAT) / 
         (SELECT COUNT(*) FROM usa_tracks) * 100, 2)
          percent_total
  FROM usa_tracks ut
 INNER JOIN track t ON t.track_id = ut.track_id
 INNER JOIN genre g ON g.genre_id = t.genre_id
 GROUP BY 1 ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percent_total
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


The most popular genre by far is "Rock", which consists of over 50% of all tracks sold in the store. There is also a reasonably-large number of tracks sold in the "Alternative & Punk" and "Metal" genres, which attract a similar sort of clientele as Rock music.

Note that 75% of all tracks sold in the store are in either the "Rock", "Alternative & Punk", or "Metal" genres, with the 14 other genres making up less than 25% of tracks sold. Perhaps the store owner should focus any advertizement campaign to make the store image appear "Rock" theme.

## 2. Best Market Countries

In this section, I will analyze customer and order statistics to help sales agents determine which country sales agents should focus their business development efforts. These factors include:
1. The countries that have the most sales, highest average sales per customer, or highest average sales per order.
2. The best customers and which country they are from.
3. The best customers by country.

First, I will take a look at the largest markets by country.

### a) Largest and Best-Developing Markets by Country

In this section, I will take a look at the customers in each country and determine how much sales revenue is generated per country. First, note that I will only be looking at countries that have more than one customer. In the query below, all countries with only a single customer will be placed into an "Other" column. For every country with more than one customer, I will look at the following statistics:
- Total sales per country
- Number of customers per country
- Number of orders per country
- Average sales per customer in each given country
- Average sales per order in each given country

The first three statistics (total sales, number of customers, and number of orders) are all indicators of the market's development level. In contrast, the last two statistics (average sales per customer and average sales per order) are indicators of a market's potential. Countries with a high average sales per customer and a high average sales per order are countries sales agents should focus their business development efforts.

In [4]:
%%sql
WITH new_country AS
-- "Other" value will contain summation of information from all countries with one customer
    (
    SELECT
        CASE WHEN (SELECT COUNT(*)
                     FROM customer
                    WHERE country = c.country
                   ) = 1 THEN "Other"
             ELSE c.country
         END country,
         c.customer_id,
         i.*
      FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )
    
SELECT
    country,
    total_sales,
    num_customers,
    num_orders,
    avg_customer_sales,
    avg_order_sales
  FROM (SELECT
            country,
            ROUND(SUM(total), 2) total_sales,
            COUNT(DISTINCT customer_id) num_customers,
            COUNT(*) num_orders,
            ROUND(SUM(total) / COUNT(DISTINCT customer_id), 2) avg_customer_sales,
            ROUND(SUM(total) / COUNT(*), 2) avg_order_sales,
            CASE -- Puts the "Other" row at the bottom of the chart
                WHEN country = "Other" THEN 1
                ELSE 0
            END AS sort
          FROM new_country
         GROUP BY 1
         ORDER BY 7 ASC, 2 DESC
        );

 * sqlite:///chinook.db
Done.


country,total_sales,num_customers,num_orders,avg_customer_sales,avg_order_sales
USA,1040.49,13,131,80.04,7.94
Canada,535.59,8,76,66.95,7.05
Brazil,427.68,5,61,85.54,7.01
France,389.07,5,50,77.81,7.78
Germany,334.62,4,41,83.66,8.16
Czech Republic,273.24,2,30,136.62,9.11
United Kingdom,245.52,3,28,81.84,8.77
Portugal,185.13,2,29,92.57,6.38
India,183.15,2,21,91.58,8.72
Other,1094.94,15,147,73.0,7.45


It looks as though the United States, Canada, and Brazil are currently the most developed markets. They each have a large number of customers and orders. However, Canada has a very low average sales per customer, and thus is not a market that would benefit significantly from additional business development. The sales agents should focus business development efforts on countries with customers that purchase a large quantity of product, so that they get the most sales revenue per customer.

In this regard, the best market to focus on by far is the Czech Republic. It has both the highest avg_customer_sales and the highest avg_order_sales. Most noteably, average customer from the Czech Republic purchases 136.62 dollars worth of music, which is more than forty dollars more than the average customer from the country with the next highest avg_customer_sales (Portugal). Since Czech customers are highly likely to spend large amounts of money on music purchases, Chinook sales agents should focus more effort on selling music to Czech customers.

### b) Best Chinook Customers

Next, I will look at the customers who have spent the most purchasing music from the Chinook store.

In [5]:
%%sql
SELECT
    c.first_name || " " || c.last_name customer_name,
    c.country,
    ROUND(SUM(i.total), 2) total_purchased
  FROM customer c
  LEFT JOIN invoice i ON i.customer_id = c.customer_id
 GROUP BY 1 ORDER BY 3 DESC, 1
 LIMIT 15

 * sqlite:///chinook.db
Done.


customer_name,country,total_purchased
František Wichterlová,Czech Republic,144.54
Helena Holý,Czech Republic,128.7
Hugh O'Reilly,Ireland,114.84
Manoj Pareek,India,111.87
Luís Gonçalves,Brazil,108.9
Fernanda Ramos,Brazil,106.92
João Fernandes,Portugal,102.96
François Tremblay,Canada,99.99
Wyatt Girard,France,99.99
Enrique Muñoz,Spain,98.01


The top two highest-spending customers are both from the Czech Republic. There are only two customers from the Czech Republic in the database, so this explains why the avg_customer_sales is so high for the Czech Republic. While it's possible that the two highest_spending customers are from the Czech Republic by happenstance, it's also possible that Czech people purchase more music (likely Rock) than people from other countries. Therefore, this is further proof that the Czech Republic is a great market for sales associates at Chinook to focus their efforts. 

### c) Best Chinook Customer by Country

Finally, I will find highest-spending customer per country.

In [6]:
%%sql
WITH
    -- Finds the total amount each customer purchased
    total_per_customer AS
        (SELECT
            c.customer_id,
            c.country country,
            SUM(i.total) total_purchased
           FROM customer c
          INNER JOIN invoice i ON i.customer_id = c.customer_id
          GROUP BY 1
        ),
    
    -- Finds the customer that spent the most per country
    max_customer AS
        (SELECT
             customer_id,
             country,
             MAX(total_purchased) total_purchased
           FROM total_per_customer
          GROUP BY 2 
        ),
    
    -- Finds other customer information for each customer in max_customer
    customer_info AS
        (SELECT 
             c.country,
             c.first_name,
             c.last_name,
             mc.total_purchased
           FROM customer c
          INNER JOIN max_customer mc ON mc.customer_id = c.customer_id
        )
         
SELECT
    country,
    first_name || " " || last_name best_customer,
    ROUND(total_purchased, 2) customer_purchased
  FROM customer_info

 * sqlite:///chinook.db
Done.


country,best_customer,customer_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.39
Brazil,Luís Gonçalves,108.9
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02
Czech Republic,František Wichterlová,144.54
Denmark,Kara Nielsen,37.62
Finland,Terhi Hämäläinen,79.2


Chinook sells music to customers in 24 different countries, 15 of which only have a single customer.

## 2. Sales by Employee

Now I will examine how successful each sales agent at Chinook is.

In [7]:
%%sql
WITH
    customer_total AS
    (
     SELECT
        c.*,
        SUM(i.total) total
       FROM customer c
     INNER JOIN invoice i ON i.customer_id = c.customer_id
     GROUP BY 1
    )

SELECT
    e.first_name || " " || e.last_name employee_name,
    e.hire_date,
    ROUND(SUM(ct.total), 2) employee_total
  FROM employee e
 INNER JOIN customer_total ct ON ct.support_rep_id = e.employee_id
 GROUP BY 1 ORDER BY 3 DESC

 * sqlite:///chinook.db
Done.


employee_name,hire_date,employee_total
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


The longer the tenure at the company, the more sales revenue the employee has generated for the company. Therefore, it is difficult to measure how well each sales agent is performing on a day to day basis.

## 3. Albums vs Individual Tracks

Next, I will whether customer purchase most of their music tracks individually or as part of an album.

In [8]:
%%sql
WITH
    invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    ),
    
    album_yes AS
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END album_purchase
        FROM invoice_first_track ifs
    )
    

SELECT
    album_purchase,
    COUNT(invoice_id) num_of_invoices,
    ROUND(CAST(count(invoice_id) AS FLOAT)
    / (SELECT COUNT(*) FROM invoice) * 100, 1)
    percent_total
 FROM album_yes
GROUP BY 1;

 * sqlite:///chinook.db
Done.


album_purchase,num_of_invoices,percent_total
no,500,81.4
yes,114,18.6


It appears as though a fifth of the music at Chinook is sold as part of an album, whereas four-fifths of the music is sold individually.